# grab GridSat fields for the bounding box +/- 5 degrees and write out 


In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import geopandas as gp

from shapely.geometry import Point
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

import warnings
warnings.filterwarnings('ignore')

## Read all lakes in 2014-2018 
#### lakes dataframe was "improved" in LakeCaseStudy.ipynb with firsttime added

In [2]:
df = pd.read_csv('ccvls_stats_2014-2018.improved.csv')
# df

In [3]:
equatorcases = df[ abs(df.coastlat) < 10 ] # 162 of them exceeding 1 day 
eq7cases = equatorcases[ equatorcases.dur_days > 6 ]  # 39 of them exceeding a week (more than 6 in days) 
eq7cases.maxarea.size

39

# Open GridSat file

In [4]:
!ls /data/willytsai/INO_megadata_2014_2018/Gridsat-BT/

2009  2012  2015  2018			   remap_0.1deg   wget.sh
2010  2013  2016  GridSat_BTdataset.ipynb  remap_0.25deg
2011  2014  2017  index.html		   robots.txt


In [5]:
!ls /data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/*2014.01.01*

/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.00.v02r01.nc
/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.03.v02r01.nc
/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.06.v02r01.nc
/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.09.v02r01.nc
/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.12.v02r01.nc
/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.15.v02r01.nc
/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.18.v02r01.nc
/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.21.v02r01.nc


In [6]:
gs = xr.open_dataset('/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/2014/GRIDSAT-B1.2014.01.01.18.v02r01.nc')
gs

<xarray.Dataset>
Dimensions:        (lat: 2000, lon: 5143, time: 1, Ngeo: 6, sparse3ir: 526026, sparse2vs: 766751, sparse2ir: 2, sparse2wv: 2)
Coordinates:
  * lat            (lat) float32 -70.0 -69.93 -69.86 ... 69.79 69.86 69.93
  * lon            (lon) float32 -180.0 -179.9 -179.9 ... 179.8 179.9 179.9
  * time           (time) datetime64[ns] 2014-01-01T18:00:00
  * sparse2ir      (sparse2ir) int32 0 1
  * sparse2wv      (sparse2wv) int32 0 1
  * sparse2vs      (sparse2vs) int32 936 937 938 ... 10282922 10282923 10282924
  * sparse3ir      (sparse3ir) int32 1632771 1632772 1632773 ... 8776401 8781544
Dimensions without coordinates: Ngeo
Data variables: (12/25)
    satlat         (time, Ngeo) float32 ...
    satlon         (time, Ngeo) float32 ...
    satrad         (time, Ngeo) float32 ...
    satname        (time, Ngeo) |S50 ...
    b1ufile        (time, Ngeo) |S50 ...
    b1file         (time, Ngeo) |S50 ...
    ...             ...
    vschn_2        (time, sparse2vs) float32 ...
    satid_ir       (time, lat, lon) float32 ...
    satid_wv       (time, lat, lon) int8 ...
    satid_vs       (time, lat, lon) int8 ...
    satid_ir3      (sparse3ir) int8 ...
    irwin_vza_adj  (time, lat, lon) float32 ...
Attributes: (12/50)
    title:                      GridSat-B1 - Gridded Satellite (GridSat) ISCC...
    summary:                    Geostationary data are remapped to equal angl...
    Conventions:                CF-1.6
    Metadata_Conventions:       CF-1.6, Unidata Dataset Discovery v1.0, NOAA ...
    references:                 http://dx.doi.org/10.1175/2011BAMS3039.1  htt...
    source:                     ISCCP.B1U.a.FY2-E.2014.01.01.1800.CMA.gz,ISCC...
    ...                         ...
    _satid:                     _satid attributes are comma delimited arrays ...
    Satellite_Name_satid:       FY2-E,GOES-13,GOES-15,Meteosat-7,Meteosat-10,...
    B1U_File_satid:             ISCCP.B1U.a.FY2-E.2014.01.01.1800.CMA.gz,ISCC...
    IRWIN_calibration_file:     b1u.calibration.correction.IRWIN.v17
    IRWVP_calibration_version:  v01
    Corrections:                IRWIN has VZA correction, All channels have p...

In [ ]:
# list(gs.keys())

In [9]:
# Just keep the irwin_cdr and irwvp (vapor channel)

gs = xr.open_mfdataset('/data/willytsai/INO_megadata_2014_2018/Gridsat-BT/201[4,5,6,7,8]/GRIDSAT-B1*.nc',
                      drop_variables=['satlat','satlon','satrad','satname','b1ufile','b1file','calslp_irwin',
                                 'caloff_irwin','calslp_irwvp','caloff_irwvp','vis_rad_slope','vis_dc_slope',
                                 'vis_dc_offset','irwin_2','irwin_3', 'irwvp_2', 'vschn', 'vschn_2',
                                 'satid_vs', 'satid_wv', # 'irwvp',
                                 'satid_ir', 'satid_ir3', 'irwin_vza_adj',
                                 'sparse2ir','sparse2wv','sparse2vs','sparse3ir' ] )

gs

<xarray.Dataset>
Dimensions:    (lat: 2000, lon: 5143, time: 14608)
Coordinates:
  * lat        (lat) float32 -70.0 -69.93 -69.86 -69.79 ... 69.79 69.86 69.93
  * lon        (lon) float32 -180.0 -179.9 -179.9 -179.8 ... 179.8 179.9 179.9
  * time       (time) datetime64[ns] 2014-01-01 ... 2018-12-31T21:00:00
Data variables:
    irwin_cdr  (time, lat, lon) float32 dask.array<chunksize=(1, 2000, 5143), meta=np.ndarray>
    irwvp      (time, lat, lon) float32 dask.array<chunksize=(1, 2000, 5143), meta=np.ndarray>
Attributes: (12/50)
    title:                      GridSat-B1 - Gridded Satellite (GridSat) ISCC...
    summary:                    Geostationary data are remapped to equal angl...
    Conventions:                CF-1.6
    Metadata_Conventions:       CF-1.6, Unidata Dataset Discovery v1.0, NOAA ...
    references:                 http://dx.doi.org/10.1175/2011BAMS3039.1  htt...
    source:                     ISCCP.B1U.a.GOE-13.2013.12.31.2345.NOA.gz,ISC...
    ...                         ...
    _satid:                     _satid attributes are comma delimited arrays ...
    Satellite_Name_satid:       GOES-13,Meteosat-7,MTSAT-1R,FY2-E,GOES-15,Met...
    B1U_File_satid:             ISCCP.B1U.a.GOE-13.2013.12.31.2345.NOA.gz,ISC...
    IRWIN_calibration_file:     b1u.calibration.correction.IRWIN.v17
    IRWVP_calibration_version:  v01
    Corrections:                IRWIN has VZA correction, All channels have p...

In [11]:
for icase in range(len(eq7cases)): 

# Continue from where it last crashed/ended:
#    if(icase < 8): 
#        continue

# Select case and move forward
    case = eq7cases.iloc[icase]
    filename = case.filename
    print('case ',filename, icase)

    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = gs.sel(lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                time=slice(gdf.time.min(),gdf.time.max())             )

    d2.to_netcdf('/data/bmapes/LAKEBYLAKE/'+filename[0:-8]+'.GridSatIR_WV_3hourly.nc')
    

case  2018_11_20_11_lat4p616S.geojson 0
case  2018_11_16_21_lat8p412S.geojson 1
case  2018_10_22_15_lat9p781N.geojson 2
case  2018_03_19_12_lat9p318S.geojson 3
case  2017_12_01_11_lat9p845S.geojson 4
case  2017_05_28_11_lat5p291S.geojson 5
case  2017_05_09_01_lat3p355S.geojson 6
case  2017_05_08_05_lat1p160N.geojson 7
case  2017_05_06_20_lat3p790S.geojson 8
case  2017_05_06_20_lat0p918N.geojson 9
case  2017_05_06_18_lat1p979S.geojson 10
case  2017_04_29_11_lat7p001S.geojson 11
case  2017_04_29_11_lat0p732S.geojson 12
case  2017_02_28_21_lat8p115S.geojson 13
case  2017_02_27_22_lat8p004S.geojson 14
case  2017_01_07_14_lat6p991S.geojson 15
case  2016_05_02_05_lat1p551S.geojson 16
case  2016_05_01_00_lat6p733S.geojson 17
case  2016_04_29_17_lat1p821N.geojson 18
case  2016_04_15_08_lat5p477S.geojson 19
case  2016_04_10_23_lat8p819S.geojson 20
case  2015_09_17_21_lat3p507N.geojson 21
case  2015_05_07_03_lat6p414S.geojson 22
case  2015_05_06_21_lat5p111N.geojson 23
case  2015_05_05_21_lat0p2

In [12]:
!date

Thu Jun  1 21:48:20 EDT 2023


In [ ]:
# For faster access later, store a copy -- ir and wv only 

allio =  gs.sel(lat=slice(-30,30),lon=slice(30,90))
allio.to_netcdf('/data/bmapes/LAKEBYLAKE/WEIO_30S30N_3090E_2014-2018.GridSatIR_WV_3hourly.nc')

In [ ]:
!date